# SQLite starter

SQL basics in Python using SQLite3

## References

* [Chapter 13 of R4DS -- Relational data](https://r4ds.had.co.nz/relational-data.html) -- r4ds.had.co.nz
* [sqlite3](https://docs.python.org/3/library/sqlite3.html) API reference -- python.org
  * [PEP 249 - Database API Specification 2.0](https://www.python.org/dev/peps/pep-0249/) (Python Enhancement Proposal) -- python.org
  * [SQL tutorial](https://www.w3schools.com/sql/) -- w3schools.com
* [Raschka's SQLite3 notes](https://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html) -- sebastianraschka.com
* [Chapter 6, Database Design](https://learning.oreilly.com/library/view/using-sqlite/9781449394592/ch06s03.html), Kreibich (2010) -- Using SQLite
* [Brief primer on merge methods (relational algebra)](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#brief-primer-on-merge-methods-relational-algebra) with Pandas -- pandas.pydata.org
  * R has dyplr


# SQLite

An RDBMS is the most common place to find relation data.

* Some of the most common RDBMS
  * PostgreSQL (PostGIS), MySQL, Oracle (Oracle Spatial)
* [SQLite](https://sqlite.org/) is a C library for a file-based Relational Database Management System (RDBMS).
  * SQL (Structured Query Language) is the standard language for working with an RDBMS.
  * SQLite uses a dialect of SQL.
* [SQLite features](https://www.sqlite.org/features.html)
  * Transactions are "ACID" (atomic, consistent, isolated, durable)
  * That is, transactions are guaranteed to produce valid data despite errors, power failures, etc.
  * Zero configuration (no back-end server)

# Python's sqlite3

* [sqlite3](https://docs.python.org/3/library/sqlite3.html) implements a standard Python API for SQLite
  * Why sqlite3?
  * A: Some applications can use sqlite3 as a data store.
  * A: It's great for prototyping, e.g.,  before porting to a production RDBMS like PostgreSQL.

# Create a database

With SQLite, creating a database is the same as making a database connection. 

It's that easy only because SQLite doesn't need a server.

## Connect to an SQLite database

You have two choices when creating a database. 
You can either 
* provide a filename, or
* use ":memory:" to create an in-memory database
* References:
  * [In-Memory Database](https://www.sqlite.org/inmemorydb.html) -- sqlite.org
    * Database is lost after a connection closes.
  * [Single File Dababase](https://www.sqlite.org/onefile.html) -- sqlite.org
    * Single-file format (cross platform)
* When you first create an SQLite database, there will be no tables or data.

In [ ]:
import sqlite3

# Create a connection to a file-based database
# Once you create the connection, you'll be able to see the database file
# Check it out...
con = sqlite3.connect('mydb.sqlite')

In [ ]:
# Create a connection to an in-memory database
# There's no evidence of the database on the local filesystem.
con = sqlite3.connect(':memory:')

## A note on persistence


**WARNING:** An in-memory database will not persist between sessions!!

**NOTE:** With Colab, a "local" file-based database won't persist between sessions either.

**IMPORTANT:** First commit, then close

If you want your database to persist, you must:

* Commit any changes,
* Close the connection.

**WARNING:** With a production database, these 2 steps are important. Do not assume that they're automatic!

## Rollbacks

* A "rollback" is an operation that returns a database to some previous state. 
* Rollbacks are important for database integrity.
  * Rollbacks can restore a database to a clean copy even after performing erroneous operations.
  * Rollbacks are crucial for recovering from database server crashes.
    * Database servers should roll back any transaction that was active at the time of the crash.
    * This allows the database to be restored to a consistent state.
    * Rollbacks usually include transaction logs and/or multiversion concurrency controls.
* [rollback](https://en.wikipedia.org/wiki/Rollback_(data_management)) (definition) -- wikipedia
  * `ROLLBACK` is also a SQL command



# Introduction to SQL

* SQL = Structured Query Language
* [SQL on MDN](https://developer.mozilla.org/en-US/docs/Glossary/SQL) -- includes some references
* [SQL commands](https://www.w3schools.com/sql/sql_intro.asp) -- w3schools.com
* [SQL as understood by SQLite](https://www.sqlite.org/lang.html) -- sqlite.org
* [Appendix C in *Using SQLite*](https://learning.oreilly.com/library/view/using-sqlite/9781449394592/apcs01.html) (2010) by Jay Kreibach -- oreilly.com
* [Other books about SQLite](https://sqlite.org/books.html) -- sqlite.org 

# Introducing SQLite3


## Create a table

There are several steps to create a database table from scratch...

1. Create a database connection
2. Use the connection to create a "cursor" for executing commands
3. Use the cursor to execute command(s)
4. Commit any changes
5. Close the connection (if we're done)

In [ ]:
import sqlite3

# Create a connection to an in-memory database
con = sqlite3.connect(':memory:')

# Create a cursor
cur = con.cursor()

# Create a table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2016-06-10','BUY','APPL',100,24.71)")

# Save (commit) the changes
con.commit()

# We can close the connection if we are done with it.
# But with an in-memory database, the database will be lost.
con.close()

In [ ]:
# Create a table in a file-based database

# Create a connection
# This line differs from the previous cell -- here we're specifing a filename
con = sqlite3.connect('example.db')

cur = con.cursor()

# This line will throw an error if the table already exists
# cur.execute('''CREATE TABLE stocks
#            (date text, trans text, symbol text, qty real, price real)''')

# So use the next line instead
cur.execute('''CREATE TABLE IF NOT EXISTS stocks
               (date text, trans text, symbol text, qty text, price real)''')

cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# With a file-based database, we can close the connection and re-open it later.
# Just be sure any changes have been committed or they will be lost.
con.commit()
con.close()

## Retrieve the data

To retrieve data, use the cursor to execute a SELECT statement. There are several options...

* treat the cursor as an [iterator](https://github.com/jakevdp/WhirlwindTourOfPython/blob/master/10-Iterators.ipynb), 
* call the cursor’s `fetchone()` method to retrieve a single matching row, or 
* call `fetchall()` to get a list of all the matching rows.

In [ ]:
# Retrieve a row using the cursor as an iterator
con = sqlite3.connect('example.db')
cur = con.cursor()
cur.execute("SELECT * FROM stocks")
for row in cur:
  print(row)

('2006-01-05', 'BUY', 'RHAT', '100', 35.14)
('2006-01-05', 'BUY', 'RHAT', '100', 35.14)
('2006-01-05', 'BUY', 'RHAT', '100', 35.14)
('2006-01-05', 'BUY', 'RHAT', '100', 35.14)


In [ ]:
# Reuse the cursor to create another query
cur.execute("SELECT * FROM stocks")

# Execute the transaction
table = cur.fetchone()

table

('2006-01-05', 'BUY', 'RHAT', '100', 35.14)

# (Re)Introducing Pandas

Pandas provides a nice RDBMS GUI, Along with all of the other capabilities.

[pandas.read_sql()](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) API reference docs -- pandas.pydata.org

## Schema Table & Table Schema

You can query the database for the schema (i.e., setup of the database)

* Ref: [The Schema Table](https://www.sqlite.org/schematab.html)
  * Also called "sqlite_master"
  * See the reference for other names and fields
  * Note, in particular the `sql` field
* [PRAGMA statements](https://sqlite.org/pragma.html)
  * [PRAGMA table_info()](https://sqlite.org/pragma.html#pragma_table_info)

# A demo database to practice relational algebra with SQL

In [ ]:
# Recreate the student/course/enrolled database
con = sqlite3.connect(':memory:')
cur = con.cursor()

cur.execute("CREATE TABLE student (sid text, name text, major text, gpa real)")
cur.execute("CREATE TABLE course (crn text, course text, dept text, term text)")
cur.execute("CREATE TABLE enrolled (sid text, crn text, grade text)")

cur.execute('''INSERT INTO student
  (sid, name, major, gpa) VALUES
  ('0001', 'John', 'CS', NULL),
  ('0002', 'Lucy', 'DS', 4.00),
  ('0003', 'Aiden', 'CS', 3.33)''')

cur.execute('''INSERT INTO course
  (crn, course, dept, term) VALUES
  ('00234', 'Intro CS', 'CS', 'Fall2020'),
  ('00653', 'Intro DS', 'CS', 'Fall2020'),
  ('00783', 'Algorithms', 'CS', 'Fall2020'),
  ('01945', 'ML & AI', 'EE', 'Spring2021')''')

cur.execute('''INSERT INTO enrolled
  (sid, crn, grade) VALUES
  ('0002', '00653', 'A'),
  ('0002', '01945', NULL),
  ('0003', '00783', 'B+')''')

con.commit()